In [117]:
import h5py
import math
import numpy as np
from operator import itemgetter
import scipy.stats as st
from itertools import compress

In [118]:
raw = open('snakes.phy', 'r')
gendata = file.read(raw) 
gendata = gendata.split('\n')
gendata.pop(0)
gendata.pop(len(gendata)-1)
gendata

['ac1OUTG1  ATGCCACAGCTAGACATCGTACACATCCTCACTATTTACATGTGAACCTGAGTAACCCTTACCTTAATTACACTAAAAATTAAAAATCTCACATTAACTGCCAAACCAAAAAAACAGCCCCAACTAACCTCCAAACAAACAACACTACCCACACCATGAACATAAACATATTCGAACAATTCATAAGCCCAGAGCTCGCCCTCCTCCCAACAGCACCCCTATCAATTCTAATTCCTCTTTTCCTGATCCACCACAAACCTACACTTTTAGGAAACCGTATAACAACCGCCATCACTTGATTCCTGAAAATATTTATATTGAACATGACAAATCAACTTACCCCAAAAGGACAAAAATGATCCCCCCTATTAGCCAGCCTTATCCTCATTATTCTACTGTCCAACCTCCTAAGTCTTCTACCATATACCTTCACACCAACCTCCCAACTATCAATAAACATAGCCCTAGCCCTTCCCCTCTGACTAGCCACCATTATCACCGGCCTAAAAAACAAGTTTTCCCTAGCACTAGCCCACCTACTCCCAGAAGGCTCCCCAACCCCCCTAATCCCATTTATGATCTTAATTGAAACAGTTAGCCTACTAATACGACCTATCGCACTAGGAGTGCGACTCACAGCAAATATCACTGCCGGACATCTCCTTATGATCTAGATGGGAGGAGAGAGAGAGAG--CCGTGAAGATCTTGGCCAACAGAAGGCCCCATCAACACCATGCGCAGAAAACACCACTGAAAAAATCTAACACATGCTGGACAGATACAAACCTTAATTGCAGGTGTTAAGGCAAGCCACACCCTTTCCGTAGTGGGCAAA-TTTCAGTTTTACAATTTTAAGAAAACATGAT?????TTTATTTGAAAGAGCAGATAAAAACCTTTCCAGATCCAGCAAGAGAAGATTCAGATCAGAAAAGAAAATCCTGCAGAGCTGGGGGAATAAAAGGAAGAAAGAAAGAAAGAAAAA

In [254]:
names = []
seqs = []
for i in range(len(gendata)):
    names_temp, seq_temp = gendata[i].split()
    names.append(names_temp)
    seqs.append(seq_temp)

In [426]:
#pick out taxa # 1 (outgroup), 10, 7, 46
hybrid = 6
parents = [2,19]
selectednames = [names[0],names[(parents[0]-1)],names[(hybrid-1)],names[(parents[1]-1)]]

#get the seqs
selectedseqs = np.array([seqs[0],seqs[(parents[0]-1)],seqs[(hybrid-1)],seqs[(parents[1]-1)]])

snps = np.empty((0,4), str)
for i in range(len(selectedseqs[0])):
    bases = [row[i] for row in selectedseqs]
    if ((len(set(bases)) > 1) & (len(set(bases) | set(['A','G','C','T'])) == 4)):
        snps = np.vstack([snps, bases])
snps




array([['G', 'G', 'A', 'A'],
       ['C', 'C', 'T', 'T'],
       ['C', 'C', 'T', 'T'],
       ..., 
       ['A', 'A', 'G', 'G'],
       ['C', 'C', 'T', 'T'],
       ['A', 'A', 'T', 'T']],
      dtype='|S1')

In [427]:
possible_configs = [[0,1,2,3],[0,2,1,3],[0,1,3,2]]

snps = np.where(snps=='A',0,snps)
snps = np.where(snps=='C',1,snps)
snps = np.where(snps=='G',2,snps)
snps = np.where(snps=='T',3,snps)
snps = snps.astype(int)
finalsnps = snps

In [428]:
# make index matrix for each pair of bases. This assigns row / col number for full 16x16 matrix
indexmat = np.array(range(16))
indexmat.shape=(4,4)

        # make 16x16 matrix of zeroes
        # order across matrix is 00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
        # not good use of space
fullmat0123 = np.zeros(shape=(16,16))
arr0123 = finalsnps[:,possible_configs[0]]
for i in range(len(arr0123)):
            # get row number 
    rownum = int(indexmat[arr0123[i][0:2][0],arr0123[i][0:2][1]])
            # get col number
    colnum = int(indexmat[arr0123[i][2:4][0],arr0123[i][2:4][1]])
    fullmat0123[rownum,colnum] = fullmat0123[rownum,colnum] + 1

fullmat0213 = np.zeros(shape=(16,16))
arr0213 = finalsnps[:,possible_configs[1]]
for i in range(len(arr0213)):
            # get row number 
    rownum = int(indexmat[arr0213[i][0:2][0],arr0213[i][0:2][1]])
            # get col number
    colnum = int(indexmat[arr0213[i][2:4][0],arr0213[i][2:4][1]])
    fullmat0213[rownum,colnum] = fullmat0213[rownum,colnum] + 1
    
fullmat0132 = np.zeros(shape=(16,16))
arr0132 = finalsnps[:,possible_configs[2]]
for i in range(len(arr0132)):
            # get row number 
    rownum = int(indexmat[arr0132[i][0:2][0],arr0132[i][0:2][1]])
            # get col number
    colnum = int(indexmat[arr0132[i][2:4][0],arr0132[i][2:4][1]])
    fullmat0132[rownum,colnum] = fullmat0132[rownum,colnum] + 1

In [429]:
fullmat0123.astype(int)

array([[ 0,  1,  1,  0,  0, 23,  0,  0,  0,  0, 54,  0,  2,  1,  0, 15],
       [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [23,  0,  0,  0,  0,  0,  0,  0,  0,  1, 12,  0,  0,  1,  0, 83],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [51,  0,  4,  0,  0,  6,  0,  0,  2,  0,  0,  0,  0,  0,  0, 12],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,

In [522]:
def calcHils(invmat, Nreq = 10, returnf = False, returnp = False, returnall = False,returnnum = False):
    comb_dict = dict(zip([00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33], [0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  10, 11, 12, 13, 14, 15]))
    num_iijj = (invmat[comb_dict[00],comb_dict[11]] + invmat[comb_dict[00],comb_dict[22]] +
        invmat[comb_dict[00],comb_dict[33]] + invmat[comb_dict[11],comb_dict[00]] + invmat[comb_dict[11],comb_dict[22]] +
        invmat[comb_dict[11],comb_dict[33]] + invmat[comb_dict[22],comb_dict[00]] + invmat[comb_dict[22],comb_dict[11]] +
        invmat[comb_dict[22],comb_dict[33]] + invmat[comb_dict[33],comb_dict[00]] + invmat[comb_dict[33],comb_dict[11]] +
        invmat[comb_dict[33],comb_dict[22]])

    num_ijji = (invmat[comb_dict[01],comb_dict[10]] + invmat[comb_dict[02],comb_dict[20]] +
        invmat[comb_dict[03],comb_dict[30]] + invmat[comb_dict[10],comb_dict[01]] + invmat[comb_dict[12],comb_dict[21]] +
        invmat[comb_dict[13],comb_dict[31]] + invmat[comb_dict[20],comb_dict[02]] + invmat[comb_dict[21],comb_dict[12]] +
        invmat[comb_dict[23],comb_dict[32]] + invmat[comb_dict[30],comb_dict[03]] + invmat[comb_dict[31],comb_dict[13]] +
        invmat[comb_dict[32],comb_dict[23]])

    num_ijij = (invmat[comb_dict[01],comb_dict[01]] + invmat[comb_dict[02],comb_dict[02]] +
        invmat[comb_dict[03],comb_dict[03]] + invmat[comb_dict[10],comb_dict[10]] + invmat[comb_dict[12],comb_dict[12]] +
        invmat[comb_dict[13],comb_dict[13]] + invmat[comb_dict[20],comb_dict[20]] + invmat[comb_dict[21],comb_dict[21]] +
        invmat[comb_dict[23],comb_dict[23]] + invmat[comb_dict[30],comb_dict[30]] + invmat[comb_dict[31],comb_dict[31]] +
        invmat[comb_dict[32],comb_dict[32]])
    [num_iijj,num_ijji,num_ijij]
    if (num_ijij == 0 and num_ijji == 0):
        return("No ijij or ijji are present in data (not enough data)")
    N = sum(map(sum, invmat))
    if (N <= Nreq):
        return("Not enough snps.")
    # calculate probability, add .05 to counts in case some of them are 0
    p_iijj = (num_iijj + .05)/N
    p_ijji = (num_ijji + .05)/N
    p_ijij = (num_ijij + .05)/N
    
    if (p_ijij > max([p_iijj,p_ijji])):
        return("Parental taxa are more closely related than hybrid. Discard this.")
    
    f1 = p_iijj - p_ijij
    f2 = p_ijji - p_ijij
    if not(f2):
        p_ijji = (num_ijji + 1 + .05)/N
        f2 = p_ijji - p_ijij
    rat_f1_f2 = f1/f2

    var_f1 = (1/N) * ( p_iijj*(1-p_iijj) + p_ijij*(1-p_ijij) + 2*p_iijj*p_ijij )
    var_f2 = (1/N) * ( p_ijji*(1-p_ijji) + p_ijij*(1-p_ijij) + 2*p_ijji*p_ijij )

    cov_f1_f2 = (1/N) * ( -p_iijj*p_ijji + p_iijj*p_ijij + p_ijji*p_ijij + p_ijij*(1-p_ijij))

    H = abs(f2 * rat_f1_f2) / math.sqrt( var_f2*(rat_f1_f2**2) - 2*cov_f1_f2*rat_f1_f2 + var_f1 )
    if returnf:
        return [H, f1, f2];
    if returnp:
        return [H, p_iijj,p_ijji,p_ijij];
    if returnall:
        return [H, f1, f2, p_iijj,p_ijji,p_ijij];
    if returnnum:
        return [num_iijj,num_ijji,num_ijij];
    if(num_ijij-num_ijji == 0):
        return('*'+str(H))
    else:
        return str(H);
def calcp(z):
    p = st.norm.sf(abs(z))*2
    return p;
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [445]:
[calcHils(fullmat0123),calcHils(fullmat0213),calcHils(fullmat0132)]

['No ijij or ijji are present in data (not enough data)',
 'Parental taxa are more closely related than hybrid. Discard this.',
 'No ijij or ijji are present in data (not enough data)']

In [446]:
#pick out taxa # 1 (outgroup), 10, 7, 46

hybrid = 6
parents = [2,19]
selectednames = [names[0],names[(parents[0]-1)],names[(hybrid-1)],names[(parents[1]-1)]]

#get the seqs
selectedseqs = np.array([seqs[0],seqs[(parents[0]-1)],seqs[(hybrid-1)],seqs[(parents[1]-1)]])

snps = np.empty((0,4), str)
for i in range(len(selectedseqs[0])):
    bases = [row[i] for row in selectedseqs]
    if ((len(set(bases)) > 1) & (len(set(bases) | set(['A','G','C','T'])) == 4)):
        snps = np.vstack([snps, bases])

possible_configs = [[0,1,2,3],[0,2,1,3],[0,1,3,2]]

snps = np.where(snps=='A',0,snps)
snps = np.where(snps=='C',1,snps)
snps = np.where(snps=='G',2,snps)
snps = np.where(snps=='T',3,snps)
snps = snps.astype(int)
finalsnps = snps

# make index matrix for each pair of bases. This assigns row / col number for full 16x16 matrix
indexmat = np.array(range(16))
indexmat.shape=(4,4)

        # make 16x16 matrix of zeroes
        # order across matrix is 00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
        # not good use of space
fullmat0123 = np.zeros(shape=(16,16))
arr0123 = finalsnps[:,possible_configs[0]]
for i in range(len(arr0123)):
            # get row number 
    rownum = int(indexmat[arr0123[i][0:2][0],arr0123[i][0:2][1]])
            # get col number
    colnum = int(indexmat[arr0123[i][2:4][0],arr0123[i][2:4][1]])
    fullmat0123[rownum,colnum] = fullmat0123[rownum,colnum] + 1

fullmat0213 = np.zeros(shape=(16,16))
arr0213 = finalsnps[:,possible_configs[1]]
for i in range(len(arr0213)):
            # get row number 
    rownum = int(indexmat[arr0213[i][0:2][0],arr0213[i][0:2][1]])
            # get col number
    colnum = int(indexmat[arr0213[i][2:4][0],arr0213[i][2:4][1]])
    fullmat0213[rownum,colnum] = fullmat0213[rownum,colnum] + 1
    
fullmat0132 = np.zeros(shape=(16,16))
arr0132 = finalsnps[:,possible_configs[2]]
for i in range(len(arr0132)):
            # get row number 
    rownum = int(indexmat[arr0132[i][0:2][0],arr0132[i][0:2][1]])
            # get col number
    colnum = int(indexmat[arr0132[i][2:4][0],arr0132[i][2:4][1]])
    fullmat0132[rownum,colnum] = fullmat0132[rownum,colnum] + 1

[calcHils(fullmat0123),calcHils(fullmat0213),calcHils(fullmat0132)]


['No ijij or ijji are present in data (not enough data)',
 'Parental taxa are more closely related than hybrid. Discard this.',
 'No ijij or ijji are present in data (not enough data)']

In [447]:
results = file.read(open('results.txt', 'r')) 

In [450]:
results = results.split('\n')

In [533]:
tests = [[int(s) for s in line.split() if s.isdigit()] for line in results]
tests[0]

[3, 2, 4]

In [545]:
allresults = []
for i in tests[0:100]:
    hybrid = 2#i[0]
    parents = [34,8]#i[1:3]
    selectednames = [names[0],names[(parents[0]-1)],names[(hybrid-1)],names[(parents[1]-1)]]

    #get the seqs
    selectedseqs = np.array([seqs[0],seqs[(parents[0]-1)],seqs[(hybrid-1)],seqs[(parents[1]-1)]])

    snps = np.empty((0,4), str)
    for i in range(len(selectedseqs[0])):
        bases = [row[i] for row in selectedseqs]
        if ((len(set(bases)) > 1) & (len(set(bases) | set(['A','G','C','T'])) == 4)):
            snps = np.vstack([snps, bases])

    possible_configs = [[0,1,2,3],[0,2,1,3],[0,1,3,2]]

    snps = np.where(snps=='A',0,snps)
    snps = np.where(snps=='C',1,snps)
    snps = np.where(snps=='G',2,snps)
    snps = np.where(snps=='T',3,snps)
    snps = snps.astype(int)
    finalsnps = snps

    # make index matrix for each pair of bases. This assigns row / col number for full 16x16 matrix
    indexmat = np.array(range(16))
    indexmat.shape=(4,4)

            # make 16x16 matrix of zeroes
            # order across matrix is 00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33
            # not good use of space
    fullmat0123 = np.zeros(shape=(16,16))
    arr0123 = finalsnps[:,possible_configs[0]]
    for i in range(len(arr0123)):
                # get row number 
        rownum = int(indexmat[arr0123[i][0:2][0],arr0123[i][0:2][1]])
                # get col number
        colnum = int(indexmat[arr0123[i][2:4][0],arr0123[i][2:4][1]])
        fullmat0123[rownum,colnum] = fullmat0123[rownum,colnum] + 1

    fullmat0213 = np.zeros(shape=(16,16))
    arr0213 = finalsnps[:,possible_configs[1]]
    for i in range(len(arr0213)):
                # get row number 
        rownum = int(indexmat[arr0213[i][0:2][0],arr0213[i][0:2][1]])
                # get col number
        colnum = int(indexmat[arr0213[i][2:4][0],arr0213[i][2:4][1]])
        fullmat0213[rownum,colnum] = fullmat0213[rownum,colnum] + 1

    fullmat0132 = np.zeros(shape=(16,16))
    arr0132 = finalsnps[:,possible_configs[2]]
    for i in range(len(arr0132)):
                # get row number 
        rownum = int(indexmat[arr0132[i][0:2][0],arr0132[i][0:2][1]])
                # get col number
        colnum = int(indexmat[arr0132[i][2:4][0],arr0132[i][2:4][1]])
        fullmat0132[rownum,colnum] = fullmat0132[rownum,colnum] + 1

    allhils = [calcHils(fullmat0123),calcHils(fullmat0213),calcHils(fullmat0132)]
    fil = [isfloat(element.replace('*','')) for element in allhils]
    hilsnumeric = list(compress(allhils,fil))
    if (len(hilsnumeric) > 0):
        printed_results = max(hilsnumeric)
    else:
        printed_results = ("No answer")
    allresults.append(printed_results)

'0.951998989458'

In [546]:
allhils

['Parental taxa are more closely related than hybrid. Discard this.',
 '0.948420045888',
 '0.951998989458']

In [530]:
temphilslist = [isfloat(element.replace('*','')) for element in allhils]
temphilslist

[True, False, True]

In [535]:
allresults

['*0.568810941416',
 '*0.568810941416',
 '1.31990884638',
 '1.31990884638',
 '1.32047676762',
 '1.32047676762',
 '1.32029596423',
 '1.32029596423',
 '1.32047676762',
 '1.32047676762',
 '1.32029596423',
 '1.32029596423',
 '1.32029596423',
 '1.32047676762',
 '1.32029596423',
 '1.32029596423',
 '1.32010685047',
 '1.32029596423',
 '1.32047676762',
 '1.32029596423',
 '1.83404108078',
 '2.04071912336',
 '1.59707470876',
 '1.83495569794',
 '1.83308544464',
 '1.83404108078',
 '1.59656626866',
 '1.83308544464',
 '1.59603445133',
 '1.40342185034',
 '1.59547761659',
 '1.59656626866',
 '1.59707470876',
 '1.40362230559',
 '1.59603445133',
 '1.13027515363',
 '1.59603445133',
 '1.13027515363',
 '1.32047676762',
 '1.32047676762',
 '1.32029596423',
 '1.59707470876',
 '1.32029596423',
 '1.32029596423',
 '1.32047676762',
 '1.32047676762',
 '1.3206497916',
 '1.59847415778',
 '1.3206497916',
 '1.3206497916',
 '0.987792339517',
 '0.987792339517',
 '0.987812966947',
 '0.987812966947',
 '0.987806696767',
 '0.

In [503]:
    invmat = fullmat0132
    comb_dict = dict(zip([00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33], [0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  10, 11, 12, 13, 14, 15]))
    num_iijj = (invmat[comb_dict[00],comb_dict[11]] + invmat[comb_dict[00],comb_dict[22]] +
        invmat[comb_dict[00],comb_dict[33]] + invmat[comb_dict[11],comb_dict[00]] + invmat[comb_dict[11],comb_dict[22]] +
        invmat[comb_dict[11],comb_dict[33]] + invmat[comb_dict[22],comb_dict[00]] + invmat[comb_dict[22],comb_dict[11]] +
        invmat[comb_dict[22],comb_dict[33]] + invmat[comb_dict[33],comb_dict[00]] + invmat[comb_dict[33],comb_dict[11]] +
        invmat[comb_dict[33],comb_dict[22]])

    num_ijji = (invmat[comb_dict[01],comb_dict[10]] + invmat[comb_dict[02],comb_dict[20]] +
        invmat[comb_dict[03],comb_dict[30]] + invmat[comb_dict[10],comb_dict[01]] + invmat[comb_dict[12],comb_dict[21]] +
        invmat[comb_dict[13],comb_dict[31]] + invmat[comb_dict[20],comb_dict[02]] + invmat[comb_dict[21],comb_dict[12]] +
        invmat[comb_dict[23],comb_dict[32]] + invmat[comb_dict[30],comb_dict[03]] + invmat[comb_dict[31],comb_dict[13]] +
        invmat[comb_dict[32],comb_dict[23]])

    num_ijij = (invmat[comb_dict[01],comb_dict[01]] + invmat[comb_dict[02],comb_dict[02]] +
        invmat[comb_dict[03],comb_dict[03]] + invmat[comb_dict[10],comb_dict[10]] + invmat[comb_dict[12],comb_dict[12]] +
        invmat[comb_dict[13],comb_dict[13]] + invmat[comb_dict[20],comb_dict[20]] + invmat[comb_dict[21],comb_dict[21]] +
        invmat[comb_dict[23],comb_dict[23]] + invmat[comb_dict[30],comb_dict[30]] + invmat[comb_dict[31],comb_dict[31]] +
        invmat[comb_dict[32],comb_dict[32]])
    [num_iijj,num_ijji,num_ijij]
    #if (num_ijij == 0 and num_ijji == 0):
    #    return("No ijij or ijji are present in data (not enough data)")
    N = sum(map(sum, invmat))
    #if (N <= Nreq):
    #    return("Not enough snps.")
    # calculate probability, add .05 to counts in case some of them are 0
    p_iijj = (num_iijj + .05)/N
    p_ijji = (num_ijji + .05)/N
    p_ijij = (num_ijij + .05)/N
    
    #if (p_ijij > max([p_iijj,p_ijji])):
    #    return("Parental taxa are more closely related than hybrid. Discard this.")
    
    f1 = p_iijj - p_ijij
    f2 = p_ijji - p_ijij
    if not(f2):
        p_ijji = (num_ijji + 1 + .05)/N
        f2 = p_ijji - p_ijij
    rat_f1_f2 = f1/f2

    var_f1 = (1/N) * ( p_iijj*(1-p_iijj) + p_ijij*(1-p_ijij) + 2*p_iijj*p_ijij )
    var_f2 = (1/N) * ( p_ijji*(1-p_ijji) + p_ijij*(1-p_ijij) + 2*p_ijji*p_ijij )

    cov_f1_f2 = (1/N) * ( -p_iijj*p_ijji + p_iijj*p_ijij + p_ijji*p_ijij + p_ijij*(1-p_ijij))

    H = abs(f2 * rat_f1_f2) / math.sqrt( var_f2*(rat_f1_f2**2) - 2*cov_f1_f2*rat_f1_f2 + var_f1 )

In [513]:
num_ijji

1.0